In [1]:
import pandas as pd

# train and test datasets
df_train = pd.read_csv("final_match_pairs_train.txt", sep = '\t') # inferred match pairs
df_test = pd.read_csv("final_match_pairs_ground_truth_additional.txt", sep = '\t') # ground-truth match pairs

# retain test dataset for testing reidentification algorithm
data_test = df_test.copy(deep = True)

# filter rows with match = 1
df_train = df_train[df_train.match == 1]
df_test = df_test[df_test.match == 1]

# drop redundant columns
index_cols = ['file', 'adv', 'stop']
df_train.drop(index_cols + ['match'], axis = 1, inplace = True)
df_test.drop(index_cols + ['match'], axis = 1, inplace = True)

# test dataset of candidate adv where travel time are to be predicted
X_adv = data_test.drop(index_cols + ['match', 'travel_time'], axis = 1)

# split training features & target into train and test sets
random_state = 42
X_train = df_train.drop('travel_time', axis = 1)
y_train = df_train.travel_time
X_test = df_test.drop('travel_time', axis = 1)
y_test = df_test.travel_time

print(f"Train dataset size: {len(X_train)}")
print(f"Test dataset size: {len(X_test)}")

Train dataset size: 5391
Test dataset size: 619


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
import tensorflow as tf
import random

# Set seeds for reproducibility
random_state = 42
np.random.seed(random_state)
tf.random.set_seed(random_state)
random.seed(random_state)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_adv = scaler.transform(X_adv)

# Define a wrapper class for FCNN
class FCNNWrapper:
    def __init__(self, units=(64, 32), dropout=0.2, learning_rate=0.001):
        self.units = units
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.model = None

    def build_model(self):
        model = tf.keras.Sequential()
        for unit in self.units:
            model.add(tf.keras.layers.Dense(unit, activation='relu'))
            model.add(tf.keras.layers.Dropout(self.dropout))
        model.add(tf.keras.layers.Dense(1, activation='linear'))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss='mse',
        )
        return model

    def fit(self, X, y):
        self.model = self.build_model()
        self.model.fit(
            X, y,
            epochs=50,
            batch_size=32,
            verbose=0,
            validation_split=0.2,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
        )

    def predict(self, X):
        if self.model is None:
            raise ValueError("The model has not been fitted yet.")
        return self.model.predict(X).flatten()

    def score(self, X, y):
        y_pred = self.predict(X)
        return -mean_squared_error(y, y_pred)  # Negative MSE for GridSearchCV

    def get_params(self, deep=True):
        return {
            "units": self.units,
            "dropout": self.dropout,
            "learning_rate": self.learning_rate,
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

# Custom scorer for RMSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Define the parameter grid for GridSearchCV
param_grid = {
    'units': [(64, 32), (128, 64), (64, 32, 16)],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01, 0.1],
}

# Define K-Fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

# Perform grid search
grid_search = GridSearchCV(
    estimator=FCNNWrapper(),
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=2
)

grid_search.fit(X_train, y_train)

# Print the best parameters and their score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score (negative MSE):", grid_search.best_score_)

2024-12-13 15:18:53.629698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 15:18:54.123749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/catsit/miniconda3/envs/tf/lib/
2024-12-13 15:18:54.123830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/catsit/miniconda3/envs/tf/lib/
2024-12-13 15:18:54.123836: W tensorflow/compiler/tf2t

Fitting 5 folds for each of 27 candidates, totalling 135 fits


2024-12-13 15:18:54.910755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 15:18:54.910756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 15:18:55.371220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/catsit/miniconda3/envs/tf/lib/
2024-12-13 15:18:55.3

34/34 [==============================] - 0s 558us/step


2024-12-13 15:21:46.371994: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (100)
2024-12-13 15:21:46.372035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-JRCHV9T): /proc/driver/nvidia/version does not exist
2024-12-13 15:21:46.372246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Best parameters: {'dropout': 0.1, 'learning_rate': 0.001, 'units': (64, 32)}
Best cross-validation score (negative MSE): -0.7724215321160649


In [3]:
# Evaluate model performance on validation fold
rmse_valid = np.sqrt(abs(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred_final = best_model.predict(X_test)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_final))
print(f"Validation RMSE: {rmse_valid:.4f}, Test RMSE: {rmse_test:.4f}")

20/20 [==============================] - 0s 541us/step
Validation RMSE: 0.8789, Test RMSE: 0.9901


In [4]:
# make predictions on candidate adv for reidentifying algorithm
y_adv_pred = best_model.predict(X_adv)

# add predicted travel time to dataset with both 1 and 0 matches
data_pred = data_test.copy(deep = True)
data_pred['y_pred'] = y_adv_pred

# save predicted travel time values
data_pred.to_csv("predicted_travel_time/FCNN.txt", sep = '\t', index = False)

33/33 [==============================] - 0s 529us/step


In [5]:
import os

tt_thru_min, tt_thru_max = 2.5, 12 # min, max of through travel time to constrain search space

# function to process candidate match pairs
def reidentifyMatchPairs(adf, sdf, id_adv, data_pred, file):
    thru_match_initial = [] # store initial candidate match pairs of adv to stop-bar det
    
    for i in id_adv:
        adv_time = adf[adf.ID == i].TimeStamp.values[0]
        adv_lane = adf[adf.ID == i].Lane.values[0]

        # stop-bar det IDs on the same lane to look for a match
        id_stop_look = set(sdf[sdf.Lane == adv_lane].ID)

        for j in id_stop_look:
            stop_time = sdf[sdf.ID == j].TimeStamp.values[0]

            if stop_time > adv_time: # look forward in timestamp
                tt_adv_stop = (stop_time - adv_time) / np.timedelta64(1, 's') # paired travel time

                if tt_thru_min <= tt_adv_stop <= tt_thru_max:
                    # get predicted travel time for file and id_adv
                    Xi = data_pred.copy(deep = True)
                    Xi = Xi[(Xi.file == file[:-4]) & (Xi.adv == i)].reset_index(drop = True) # discard .txt
                    
                    tt_predict = Xi.loc[0, 'y_pred'] # predicted travel time
                    tt_diff = round(abs(tt_adv_stop - tt_predict), 4) # abs diff between paired & predicted

                    # store adv ID, stop ID, travel time diff
                    thru_match_initial.append([i, j, tt_diff])

    # dicts to store the lowest error for each adv, stop ID
    seen_adv_id, seen_stop_id = {}, {}

    # iterate through each candidate pair
    for pair in thru_match_initial:
        adv_id, stop_id, error = pair

        # check if adv ID not seen or if error is lower than seen error for that adv ID
        if (adv_id not in seen_adv_id) or (error < seen_adv_id[adv_id][1]):
            seen_adv_id[adv_id] = list([stop_id, error])

        # check if stop ID not seen or if error is lower than seen error for that stop ID
        if (stop_id not in seen_stop_id) or (error < seen_stop_id[stop_id][1]):
            seen_stop_id[stop_id] = list([adv_id, error])

    # match pairs for adv with lowest error
    df_adv = pd.DataFrame(seen_adv_id, index = ['adv', 'stop']).T.reset_index()
    df_adv.columns = ['adv', 'stop', 'error']

    # match pairs for stop with lowest error
    df_stop = pd.DataFrame(seen_stop_id, index = ['stop', 'adv']).T.reset_index()
    df_stop.columns = ['stop', 'adv', 'error']
    
    return {'df_adv': df_adv, 'df_stop': df_stop}

file_path = "data"
files = os.listdir(file_path)  # list of processed files to run through reidentifying algorithm

df_result = [] # store reidentified match pairs from each file

for file in files:
    print("Running reidentification algorithm for file: ", file)
    # read events-processed file with timestamp data
    df = pd.read_csv(os.path.join(file_path, file), sep = '\t')
    df.TimeStamp = pd.to_datetime(df.TimeStamp, format = '%Y-%m-%d %H:%M:%S.%f').sort_values()
    df.dropna(axis = 0, inplace = True) # drop rows with Nan

    # data frames for adv and stop-bar det
    adf = df[df.Det == 'adv']
    sdf = df[df.Det == 'stop']
    id_adv = list(sorted(adf.ID))

    # process candidate match pairs to get datasets of adv and stop pairs
    candidate_match_result = reidentifyMatchPairs(adf, sdf, id_adv, data_pred, file)
    df_adv = candidate_match_result['df_adv']
    df_stop = candidate_match_result['df_stop']

    # resulting common match pairs
    df_match_pair = df_adv.merge(df_stop, on = ['adv', 'stop', 'error'])
    df_match_pair['file'] = file[:-4]
    df_result.append(df_match_pair)

match_result = pd.concat(df_result)
match_result.to_csv("reidentification_result/FCNN.txt", sep = '\t')

# ground-truth match pairs for index cols
match_ground = data_test.copy(deep = True)
num_candidate_pairs = match_ground.shape[0]
print(f"\nNum of candidate pairs: {num_candidate_pairs}\n")

# filter ground-truth match pairs for match == 1 and select index cols
match_ground = match_ground[match_ground.match == 1][index_cols]

# get true positive (TP), false positive (FP), and false negative (FN) matches   
match_TP = pd.merge(match_result, match_ground, on = index_cols)
match_FP = match_result.merge(match_ground, on = index_cols, how = 'left', indicator = True).query('_merge == "left_only"').drop(columns = '_merge')
match_FN = match_ground.merge(match_result, on = index_cols, how = 'left', indicator = True).query('_merge == "left_only"').drop(columns = '_merge')

# num of TP, FP, FN
TP, FP, FN = match_TP.shape[0], match_FP.shape[0], match_FN.shape[0]
TN = num_candidate_pairs - TP - FP - FN

# compute metrics
accuracy = round((TP + TN) / (TP + FP + FN + TN), 4)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2*precision*recall / (precision + recall)

print(f"TP, FP, FN: {TP}, {FP}, {FN}")
print(f"Accuracy, Precision, Recall, F1: {accuracy:.4f}, {precision:.4f}, {recall:.4f}, {f1:.4f}")

Running reidentification algorithm for file:  20230327_0700_1400.txt
Running reidentification algorithm for file:  20221206_0945_1200.txt
Running reidentification algorithm for file:  20221214_0645_0715.txt
Running reidentification algorithm for file:  20230327_1415_1900.txt
Running reidentification algorithm for file:  20221206_0845_0915.txt
Running reidentification algorithm for file:  20221214_0945_1015.txt

Num of candidate pairs: 1040

TP, FP, FN: 532, 31, 87
Accuracy, Precision, Recall, F1: 0.8865, 0.9449, 0.8595, 0.9002
